In [39]:
import json
import itertools
import urllib
from tqdm.notebook import tqdm
from __future__ import print_function

import rdflib
from rdflib import Namespace
from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, SKOS, OWL, RDF, RDFS, VOID, XMLNS, XSD

In [25]:
# https://developers.google.com/knowledge-graph/

root = r"C:\Users\Simon\Desktop\SAFRANExplorer\labelgraph\Labels"
train_path = r"C:\Users\Simon\Desktop\SAFRANExplorer\workspace\data\FB15-237\train.txt"
test_path = r"C:\Users\Simon\Desktop\SAFRANExplorer\workspace\data\FB15-237\test.txt"
valid_path = r"C:\Users\Simon\Desktop\SAFRANExplorer\workspace\data\FB15-237\valid.txt"

In [45]:
def add_node(node):
    nodes.add(node)

def read_nodes(path):
    content = None
    with open(path) as f:
        content = f.readlines()
    content = [x.strip() for x in content]

    for line in content:
        head,rel,tail = line.split("\t")
        add_node(head)
        add_node(tail)
        
def divide_chunks(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]
        
def load_json_file(path):
    with open(path) as json_file:
        return json.load(json_file)
        

In [46]:
nodes = set()
read_nodes(train_path)
read_nodes(test_path)
read_nodes(valid_path)
nodes = list(nodes)

In [42]:
print(len(nodes))


14541


In [41]:

# api_key = open('.api_key').read()
api_key = ""

ids = ["/m/08966", "/m/05lf_"]
query = 'Taylor Swift'
service_url = 'https://kgsearch.googleapis.com/v1/entities:search'

limit = 500

result = {
    'result': []
}

for node_list in divide_chunks(nodes, limit):
    params = [
        ('limit', limit),
        ('indent', True),
        ('key', api_key),
    ]
    params.extend(zip(itertools.repeat('ids'), node_list))
    
    url = service_url + '?' + urllib.parse.urlencode(params)
    response = json.loads(urllib.request.urlopen(url).read())
    
    result['result'].extend(response['itemListElement'])
        
with open(root + "//fb.json", 'w') as outfile:
    json.dump(result, outfile, indent=4, sort_keys=True)

In [54]:
kg = Namespace("http://g.co/kg")

g = rdflib.Graph()

g.bind("kg", kg)
g.bind("rdfs", RDFS)

In [55]:

json_path = root + "//fb.json"
jsonfile = load_json_file(json_path)
for x in tqdm(jsonfile["result"]):
    term = x["result"]["@id"].replace("kg:","")
    g.add((
        kg.term(term),
        RDFS.label,
        rdflib.Literal(x["result"]["name"], datatype=XSD.string)

    ))
    if "detailedDescription" in x["result"]:
        g.add((
            kg.term(term),
            RDFS.comment,
            rdflib.Literal(x["result"]["detailedDescription"]["articleBody"], datatype=XSD.string)
        ))
    elif "description" in x["result"]:
        g.add((
            kg.term(term),
            RDFS.comment,
            rdflib.Literal(x["result"]["description"], datatype=XSD.string)
        ))
    
g.serialize(r"C:\Users\Simon\Desktop\SAFRANExplorer\labelgraph\Graphs\fb15k237.xml",format="xml")

0%|          | 0/13954 [00:00<?, ?it/s]